In [ ]:
from gensim.parsing.preprocessing import *

In [ ]:
FILTERS = [
    stem_text,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    remove_stopwords,
]

In [ ]:
import pandas as pd

post_df = pd.read_json("data/preprocessed_data.jsonl", orient="records")

In [ ]:
flattened = []
for tags in post_df.head().tags.to_list():
    flattened += tags
flattened

In [ ]:
from collections import OrderedDict

canonical = OrderedDict()
for word in flattened:
    p_word = ' '.join(preprocess_string(word, FILTERS))
    if p_word not in canonical:
        canonical[p_word] = [word]
    else:
        canonical[p_word].append(word)
canonical

In [ ]:
import numpy as np
for x in np.mean(np.array([[1,2,3],[2,1,3]]), axis=0):
    print(x)

In [ ]:
from transformers import AutoTokenizer, BertConfig, BertForMaskedLM

from txtai.pipeline import HFTrainer

config = BertConfig(
    vocab_size = 500,
    hidden_size = 50,
    num_hidden_layers = 2,
    num_attention_heads = 2,
    intermediate_size = 100,
)

model = BertForMaskedLM(config)
model.save_pretrained("bert")
tokenizer = AutoTokenizer.from_pretrained("bert")

train = HFTrainer()

# Train model
train((model, tokenizer), dataset, task="language-modeling", output_dir="bert",
      fp16=True, per_device_train_batch_size=128, num_train_epochs=10,
      dataloader_num_workers=2)